# Ensemble Learning

## Initial Imports

In [187]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [188]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [189]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [190]:
# Load the data
file_path = Path('../Data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [191]:
# Create our features
X = df.drop(columns=['loan_status', 'next_pymnt_d', 'issue_d'])

# Create our target
y = df['loan_status'].ravel()

In [192]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [193]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [194]:
# Convert string values to binary sets
X_binary = pd.get_dummies(X, columns=['home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'])

In [195]:
# Scale data using StandardScaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_binary)
X_scaled = ss.transform(X_binary)

StandardScaler()

In [196]:
X_scaled_df = pd.DataFrame(X_scaled, columns=X_binary.columns)

In [197]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, random_state=1)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [198]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=142, random_state=1).fit(X_train, y_train)

In [199]:
# Calculated the balanced accuracy score
from sklearn.metrics import  balanced_accuracy_score
predictions_brf = brf.predict(X_test)
balanced_accuracy_score(y_test, predictions_brf)

0.7078108068056572

In [200]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions_brf)

array([[   60,    41],
       [ 3052, 14052]], dtype=int64)

In [201]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.59      0.82      0.04      0.70      0.48       101
   low_risk       1.00      0.82      0.59      0.90      0.70      0.50     17104

avg / total       0.99      0.82      0.60      0.90      0.70      0.50     17205



In [208]:
# List the features sorted in descending order by feature importance
features = pd.DataFrame(brf.feature_importances_).T
features.columns = X_scaled_df.columns
features.sort_values(0, ascending=False, axis=1)#.to_csv(Path('../Data/features.csv'))

,last_pymnt_amnt,total_rec_prncp,total_rec_int,total_pymnt,total_pymnt_inv,int_rate,dti,installment,mths_since_recent_inq,mths_since_rcnt_il,...,num_tl_30dpd,num_tl_120dpd_2m,delinq_amnt,pymnt_plan_n,acc_now_delinq,policy_code,collection_recovery_fee,recoveries,hardship_flag_N,debt_settlement_flag_N
0,0.073315,0.072789,0.057359,0.049839,0.04861,0.030849,0.018803,0.01794,0.017052,0.016774,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Easy Ensemble Classifier

In [203]:
# Train the Classifier
from imblearn.ensemble import  EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=142, random_state=1).fit(X_train, y_train)

In [204]:
# Calculated the balanced accuracy score
predictions_eec = eec.predict(X_test)
balanced_accuracy_score(y_test, predictions_eec)

0.7301291342885458

In [205]:
# Display the confusion matrix
confusion_matrix(y_test, predictions_eec)

array([[   70,    31],
       [ 3982, 13122]], dtype=int64)

In [207]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.77      0.03      0.73      0.53       101
   low_risk       1.00      0.77      0.69      0.87      0.73      0.54     17104

avg / total       0.99      0.77      0.69      0.86      0.73      0.54     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The EasyEnsemblerClassifier model which was surprising hence the name and the robustness the BalancedRandomForestClassifier seems to imply

2. Which model had the best recall score?

    Again the EasyEnsemblerClassifier came out on top with a significant margin in the high-risk area. Which in my opinion is more important when it comes to recall

3. Which model had the best geometric mean score?

    Not by much but EasyEnsemblerClassifier seems to be the winner

4. What are the top three features?

    This is where it gets interesting...
    So top three would be ```['last_pymnt_amnt',  'total_rec_prncp', 'total_rec_int']``` which implies that volume of the last payment, magnitude of the total recent principle, as well as the total recent interest seems to hold the most weight.
    Yet! I hav noticed there are some features such as ``` 'delinq_amnt' ``` and ``` 'acc_now_delinq' ``` which I personally thought would draw more weight to the risk of a loan.
    Of course you have some irrelevant data that refers to policy code and such but none the less I did not want to omit too many features.